# 如何将人机交互流程添加到预构建的 ReAct 代理中

本教程将展示如何将人机交互流程添加到预构建的 ReAct 代理中。请参阅 [本教程](./create-react-agent.ipynb) 了解如何开始使用预构建的 ReAct 代理

您可以通过将 `interruptBefore: ["tools"]` 传递给 `createReactAgent` 在调用工具之前添加断点。请注意，您需要使用检查指针才能使其工作。

## 设置

首先，我们需要安装所需的软件包。
```bash
yarn add @langchain/langgraph @langchain/openai @langchain/core
```
本指南将使用 OpenAI 的 GPT-4o 模型。我们可以选择设置 API 密钥
对于 [LangSmith 追踪](https://smith.langchain.com/)，这将为我们提供
一流的可观测性。

In [1]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__..."
// process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "ReAct Agent with human-in-the-loop: LangGraphJS";

ReAct Agent with human-in-the-loop: LangGraphJS


## 代码

现在我们可以使用预构建的 `createReactAgent` 函数来设置我们的代理并进行人机交互：

In [70]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from '@langchain/core/tools';
import { z } from 'zod';
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { MemorySaver } from "@langchain/langgraph";

const model = new ChatOpenAI({
    model: "gpt-4o",
  });

const getWeather = tool((input) => {
    if (['sf', 'san francisco'].includes(input.location.toLowerCase())) {
        return 'It\'s always sunny in sf';
    } else if (['nyc', 'new york city'].includes(input.location.toLowerCase())) {
        return 'It might be cloudy in nyc';
    }
    else {
        throw new Error("Unknown Location");
    }
}, {
    name: 'get_weather',
    description: 'Call to get the current weather in a given location.',
    schema: z.object({
        location: z.string().describe("Location to get the weather for."),
    })
})

// 这里我们只保存在内存中
const memory = new MemorySaver();

const agent = createReactAgent({ llm: model, tools: [getWeather], interruptBefore: ["tools"], checkpointSaver: memory });


## 用法

In [71]:
let inputs = { messages: [{ role: "user", content: "what is the weather in SF california?" }] };
let config = { configurable: { thread_id: "1" } };

let stream = await agent.stream(inputs, {
  ...config,
  streamMode: "values",
});

for await (
  const { messages } of stream
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  }
  if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  }
  console.log("-----\n");
}

what is the weather in SF california?
-----

[
  {
    name: 'get_weather',
    args: { location: 'SF, California' },
    type: 'tool_call',
    id: 'call_AWgaSjqaYVQN73kL0H4BNn1Q'
  }
]
-----



我们可以验证我们的图表是否停在正确的位置：

In [72]:
const state = await agent.getState(config)
console.log(state.next)

[ 'tools' ]


现在，我们可以在继续下一个节点之前批准或编辑工具调用。如果我们想批准工具调用，我们只需继续使用 `null` 输入流式传输图形即可。如果我们想编辑工具调用，我们需要更新状态以获得正确的工具调用，然后在应用更新后我们可以继续。

我们可以尝试恢复，我们会看到出现错误：

In [73]:
stream = await agent.stream(null, {
  ...config,
  streamMode: "values",
});

for await (
    const { messages } of stream
  ) {
    let msg = messages[messages?.length - 1];
    if (msg?.content) {
      console.log(msg.content);
    }
    if (msg?.tool_calls?.length > 0) {
      console.log(msg.tool_calls);
    }
    console.log("-----\n");
  }

Error: Unknown Location
 Please fix your mistakes.
-----

[
  {
    name: 'get_weather',
    args: { location: 'San Francisco, California' },
    type: 'tool_call',
    id: 'call_MfIPKpRDXRL4LcHm1BxwcSTk'
  }
]
-----



出现此错误是因为我们的工具参数“加利福尼亚州旧金山”不是我们的工具识别的位置。

让我们展示如何编辑工具调用来搜索“旧金山”而不是“加利福尼亚州旧金山” - 因为我们的工具在编写时将“加利福尼亚州旧金山”视为未知位置。我们将更新状态，然后恢复流式传输图表，并且应该不会出现错误。请注意，仅当使用具有完全相同 ID 的消息时，我们用于 `messages` 通道的减速器才会替换消息。因此，我们可以执行 `new AiMessage(...)`，而必须直接修改来自 `messages` 通道的最后一条消息，并确保不要编辑其 ID。

In [74]:
// 首先，让我们获取当前状态
const currentState = await agent.getState(config);

// 现在让我们获取该状态的最后一条消息
// 这是我们要更新的工具调用
let lastMessage = currentState.values.messages[currentState.values.messages.length - 1]

// 现在让我们更新该工具调用的参数
lastMessage.tool_calls[0].args = { location: "San Francisco" }

// 现在让我们调用“updateState”在“messages”键中传入此消息
// 这将被视为任何其他状态更新
// 它将被传递给“messages”键的reducer函数
// 该减速器函数将使用消息的 ID 来更新它
// 拥有正确的 ID 非常重要！否则它会被附加
// 作为新消息
await agent.updateState(config, { messages: lastMessage });

{
  configurable: {
    thread_id: '1',
    checkpoint_ns: '',
    checkpoint_id: '1ef6638d-bfbd-61d0-8004-2751c8c3f226'
  }
}


In [75]:
stream = await agent.stream(null, {
  ...config,
  streamMode: "values",
});

for await (
  const { messages } of stream
) {
  let msg = messages[messages?.length - 1];
  if (msg?.content) {
    console.log(msg.content);
  }
  if (msg?.tool_calls?.length > 0) {
    console.log(msg.tool_calls);
  }
  console.log("-----\n");
}

It's always sunny in sf
-----

The climate in San Francisco is sunny right now. If you need more specific details, feel free to ask!
-----



极好的！我们的图表正确更新以查询旧金山的天气并得到正确的“今天旧金山的天气晴朗！
” 该工具的响应。